In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, roc_curve, auc
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import LocalOutlierFactor
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [3]:
warnings.filterwarnings('ignore')
%matplotlib inline

In [5]:
def inject_and_label_anomalies(data: pd.DataFrame, target_anomaly_rate: float = 0.07):
    """
    Inject synthetic anomalies and label natural and synthetic anomalies in the dataset.

    Parameters:
        data (pd.DataFrame): The dataset containing 'inside_temperature' and 'inside_humidity'.
        target_anomaly_rate (float): The target percentage of synthetic anomalies to inject.

    Returns:
        tuple: Modified dataset with anomalies, DataFrame with anomaly labels.
    """
    # Set random seed for reproducibility
    np.random.seed(42)

    # Calculate dynamic thresholds using percentiles
    lower_temp_threshold = np.percentile(data['inside_temperature'], 2)
    upper_temp_threshold = np.percentile(data['inside_temperature'], 55)
    lower_hum_threshold = np.percentile(data['inside_humidity'], 30)
    upper_hum_threshold = np.percentile(data['inside_humidity'], 80)

    # Identify natural anomalies
    labels = pd.DataFrame({'anomaly': 0}, index=data.index)
    labels.loc[
        (data['inside_temperature'] < lower_temp_threshold) | (data['inside_temperature'] > upper_temp_threshold) |
        (data['inside_humidity'] < lower_hum_threshold) | (data['inside_humidity'] > upper_hum_threshold),
        'anomaly'
    ] = 1

    # Inject synthetic anomalies
    num_synthetic_anomalies = int(len(data) * target_anomaly_rate)
    synthetic_anomaly_indices = np.random.choice(
        data.index, size=num_synthetic_anomalies, replace=False
    )
    data.loc[synthetic_anomaly_indices, 'inside_temperature'] += np.random.uniform(65, 100, size=num_synthetic_anomalies)
    data.loc[synthetic_anomaly_indices, 'inside_humidity'] += np.random.uniform(-10, 20, size=num_synthetic_anomalies)

    labels.loc[synthetic_anomaly_indices, 'anomaly'] = 1
    print(labels.shape)

    return data, labels



In [7]:
def preprocess_data(data):
    """
    Preprocess data with null value handling
    """
    data['time'] = pd.to_datetime(data['time'])

    processed_data = data.copy()
    
    # Drop house column if exists
    if 'house' in processed_data.columns:
        processed_data = processed_data.drop('house', axis=1)
    
    # Time features
    processed_data['hour'] = processed_data['time'].dt.hour
    processed_data['day_of_week'] = processed_data['time'].dt.dayofweek
    processed_data['month'] = processed_data['time'].dt.month
    processed_data['day'] = processed_data['time'].dt.day
    processed_data['is_weekend'] = processed_data['day_of_week'].isin([5, 6]).astype(int)
    
    # Handle missing values
    numerical_cols = ['outside_temperature', 'outside_humidity','BP','WS','WD_Avg','WSgust_Max','Rain_mm_Tot',
                     'hour', 'day_of_week', 'month', 'day']
    
    # For numerical columns: interpolate between values, then forward/backward fill any remaining nulls
    for col in numerical_cols:
        if col in processed_data.columns:
            processed_data[col] = (processed_data[col]
                                 .interpolate(method='linear')
                                 .fillna(method='ffill')
                                 .fillna(method='bfill'))
    
    # For categorical columns: fill with mode
    categorical_cols = processed_data.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        if col != 'time':  # Skip time column
            processed_data[col] = processed_data[col].fillna(processed_data[col].mode()[0])
    
    # One-hot encode categorical features
    processed_data = pd.get_dummies(processed_data, columns=['zone', 'device_id'])
    
    # Scale numerical features
    scaler = StandardScaler()
    processed_data[numerical_cols] = scaler.fit_transform(processed_data[numerical_cols])
    
    # Verify no nulls remain
    assert processed_data.isnull().sum().sum() == 0, "Null values remain after preprocessing"
    
    return processed_data, scaler

In [9]:
def visualize_anomalies(data: pd.DataFrame, labels: pd.DataFrame):
    """
    Visualize the anomalies in the dataset.

    Parameters:
        data (pd.DataFrame): The dataset containing 'inside_temperature' and 'inside_humidity'.
        labels (pd.DataFrame): The DataFrame containing anomaly labels (1 for anomalies, 0 for normal points).
    """
    plt.figure(figsize=(14, 8))

    # Plot temperature
    plt.subplot(2, 1, 1)
    plt.scatter(data.index, data['inside_temperature'], label='Temperature', alpha=0.3, s=5, color='blue')
    
    # Highlight anomalies
    anomalies = data[labels['anomaly'] == 1]
    plt.scatter(anomalies.index, anomalies['inside_temperature'], 
                color='red', label='Anomalies', edgecolor='black', s=10)

    plt.xlabel('Index')
    plt.ylabel('Temperature')
    plt.title('Anomaly Detection in Temperature')
    plt.legend()
    plt.tight_layout()

    # Plot humidity
    plt.subplot(2, 1, 2)
    plt.scatter(data.index, data['inside_humidity'], label='Humidity', alpha=0.3, s=5, color='green')

    # Highlight anomalies
    plt.scatter(anomalies.index, anomalies['inside_humidity'], 
                color='red', label='Anomalies', edgecolor='black', s=10)

    plt.xlabel('Index')
    plt.ylabel('Humidity')
    plt.title('Anomaly Detection in Humidity')
    plt.legend()
    plt.tight_layout()

    plt.show()




In [31]:
from sklearn.model_selection import GridSearchCV

def tune_xgboost(X_train, y_train):
    """
    Perform hyperparameter tuning for XGBoost using GridSearchCV
    """
    # Define the parameter grid
    param_grid = {
        'n_estimators': [200],
        'learning_rate': [0.02],
        'max_depth': [ 6],
        'subsample': [ 1.0],
        'colsample_bytree': [ 1.0]
    }

    # Instantiate the model
    xgb_model = xgb.XGBClassifier(
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss'
    )

    # GridSearchCV for hyperparameter tuning
    grid_search = GridSearchCV(
        estimator=xgb_model,
        param_grid=param_grid,
        scoring='f1',  # Or use 'roc_auc', 'accuracy', etc.
        cv=2,
        verbose=1
    )

    # Fit the model
    grid_search.fit(X_train, y_train)

    # Print the best parameters and best score
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Best F1 Score: {grid_search.best_score_:.3f}")

    return grid_search.best_estimator_


In [13]:
def train_naive_bayes(X_train, y_train):
    """
    Train Naive Bayes classifier
    """
    model = GaussianNB()
    model.fit(X_train, y_train)
    return model


In [15]:
def create_autoencoder(input_dim):
    """
    Enhanced autoencoder with deeper architecture and dropout
    """
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    encoded = Dense(256, activation='relu')(input_layer)
    encoded = Dropout(0.2)(encoded)
    encoded = Dense(128, activation='relu')(encoded)
    encoded = Dropout(0.2)(encoded)
    encoded = Dense(64, activation='relu')(encoded)
    encoded = Dropout(0.2)(encoded)
    encoded = Dense(32, activation='relu')(encoded)
    encoded = BatchNormalization()(encoded)
    encoded = Dense(16, activation='relu')(encoded)
    
    # Decoder
    decoded = Dense(32, activation='relu')(encoded)
    decoded = BatchNormalization()(decoded)
    decoded = Dense(64, activation='relu')(decoded)
    decoded = Dropout(0.2)(decoded)
    decoded = Dense(128, activation='relu')(decoded)
    decoded = Dropout(0.2)(decoded)
    decoded = Dense(256, activation='relu')(decoded)
    decoded = Dropout(0.2)(decoded)
    decoded = Dense(input_dim, activation='sigmoid')(decoded)
    
    # Autoencoder
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder


In [29]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import LocalOutlierFactor

def train_lof_with_tuning(X_train):
    """
    Perform parameter tuning for LOF using GridSearchCV
    """
    from sklearn.neighbors import LocalOutlierFactor

    params = {
        'n_neighbors': [ 50, 100],
        'contamination': [0.05, 0.1, 0.2],
        'metric': ['euclidean']
    }

    # Initialize LOF model in novelty mode
    lof_model = LocalOutlierFactor(novelty=True)

    # Perform grid search
    grid_search = GridSearchCV(estimator=lof_model, param_grid=params, cv=2, scoring='f1', verbose=1)
    grid_search.fit(X_train, X_train)  # LOF in novelty mode fits X_train to itself

    print("Best Parameters:", grid_search.best_params_)
    print("Best F1 Score:", grid_search.best_score_)

    # Return the best model
    return grid_search.best_estimator_


In [19]:
def plot_roc_curves(y_test, predictions_dict):
    """
    Plot ROC curves for all models
    """
    plt.figure(figsize=(10, 8))
    
    for model_name, y_pred_proba in predictions_dict.items():
        if y_pred_proba is not None:  # Some models might not have probability predictions
            fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
            roc_auc = auc(fpr, tpr)
            plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.2f})')
    
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for Different Models')
    plt.legend(loc="lower right")
    plt.show()


In [33]:
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

def create_autoencoder_tuning(input_dim, encoding_dims, dropout_rate, learning_rate, activation):
    """
    Parametrized autoencoder creation function for tuning
    """
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    encoded = Dense(encoding_dims[0], activation=activation)(input_layer)
    encoded = Dropout(dropout_rate)(encoded)
    for dim in encoding_dims[1:-1]:
        encoded = Dense(dim, activation=activation)(encoded)
        encoded = Dropout(dropout_rate)(encoded)
        encoded = BatchNormalization()(encoded)
    encoded = Dense(encoding_dims[-1], activation=activation)(encoded)
    
    # Decoder
    decoded = encoded
    for dim in reversed(encoding_dims[:-1]):
        decoded = Dense(dim, activation=activation)(decoded)
        decoded = Dropout(dropout_rate)(decoded)
        decoded = BatchNormalization()(decoded)
    decoded = Dense(input_dim, activation='linear')(decoded)  # Output activation should match the problem
    
    # Autoencoder
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    
    return autoencoder


def train_and_evaluate_autoencoder(X_train, X_test, y_test):
    """
    Perform hyperparameter tuning and train the final model
    """
    # Define parameter space
    param_grid = {
    'model__encoding_dims': [[256, 128, 64, 32, 16]],
    'model__dropout_rate': [0.1],
    'model__learning_rate': [0.001],
    'model__activation': ['relu'],
    'batch_size': [128],
    'epochs': [50]
    }

    
    # Create model for tuning with scikeras
    model = KerasRegressor(
        model=create_autoencoder_tuning,  
        input_dim=X_train.shape[1],       
        encoding_dims=[256, 128, 64, 32, 16], 
        dropout_rate=0.2,
        learning_rate=0.001,
        activation='relu',                
        epochs=50,
        batch_size=32,
        verbose=0
    )

    
    # Early stopping callback
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )
    
    # Random search
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=1,
        cv=2,
        scoring='neg_mean_squared_error',
        n_jobs=1, 
        verbose=1
    )
    
    print("Starting hyperparameter tuning...")
    random_search.fit(X_train, X_train)
    
    # Get best parameters
    best_params = random_search.best_params_
    print("\nBest parameters found:")
    for param, value in best_params.items():
        print(f"{param}: {value}")
    
    # Extract model parameters from best_params
    model_params = {k.replace('model__', ''): v for k, v in best_params.items() 
                   if k.startswith('model__')}
    
    # Train final model with best parameters
    print("\nTraining final model with best parameters...")
    final_model = create_autoencoder_tuning(
        X_train.shape[1],
        **model_params
    )
    
    history = final_model.fit(
        X_train, X_train,
        epochs=best_params['epochs'],
        batch_size=best_params['batch_size'],
        validation_split=0.2,
        callbacks=[early_stopping],
        verbose=1
    )
    
    # Evaluate model
    reconstructed = final_model.predict(X_test)
    mse = np.mean(np.power(X_test - reconstructed, 2), axis=1)
    threshold = np.percentile(mse, 90)
    ae_pred = (mse > threshold).astype(int)
    
    # Plot training history
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss During Training')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()
    
    # Evaluate and return results
    metrics = evaluate_model(y_test, ae_pred, mse, "Tuned Autoencoder")
    return final_model, mse, metrics

In [ ]:
!pip install scikeras

In [23]:
def evaluate_model(y_true, y_pred, y_pred_proba, model_name):


    """
    Evaluate model performance
    """
    print('y_true',y_true.shape)
    print('y_pred',y_pred.shape)
    print('y_pred_proba',y_pred_proba.shape)

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\n{model_name} Performance:")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")
    
    plt.figure(figsize=(6,4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{model_name} Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    return precision, recall, f1, y_pred_proba


In [25]:
def trainAndDetect(data):
    """
    Main execution function
    """
    print("Starting anomaly detection analysis...")
    
    # Generate anomaly labels
    print("\nGenerating anomaly labels...")
    data, y = inject_and_label_anomalies(data)
    
    # Visualize the anomalies
    print("\nVisualizing anomalies...")
    visualize_anomalies(data, y)
    
    # Preprocess data
    print("\nPreprocessing data...")
    processed_data, scaler = preprocess_data(data)
    
    # Prepare features
    X = processed_data.drop(['time'], axis=1)
    
    # Train-test split
    print("\nSplitting data into train and test sets...")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    
    # Dictionary to store predictions for ROC curves
    predictions_dict = {}
    
    # Train and evaluate models
    print("\nTraining and evaluating models...")
    
    # 1. XGBoost
    print("\nTraining XGBoost...")
    print("\nTuning XGBoost...")
    tuned_xgb_model = tune_xgboost(X_train, y_train)
    xgb_pred = tuned_xgb_model.predict(X_test)
    xgb_pred_proba = tuned_xgb_model.predict_proba(X_test)[:, 1]
    print(xgb_pred)
    print(y_test)
    xgb_metrics = evaluate_model(y_test, xgb_pred, xgb_pred_proba, "Tuned XGBoost")
    predictions_dict['Tuned XGBoost'] = xgb_pred_proba

    
    # 2. Naive Bayes
    print("\nTraining Naive Bayes...")
    nb_model = train_naive_bayes(X_train, y_train)
    nb_pred = nb_model.predict(X_test)
    nb_pred_proba = nb_model.predict_proba(X_test)[:, 1]
    nb_metrics = evaluate_model(y_test, nb_pred, nb_pred_proba, "Naive Bayes")
    predictions_dict['Naive Bayes'] = nb_pred_proba
    
    # 3. Autoencoder
    #print("\nTraining Autoencoder...") 
    #autoencoder = create_autoencoder(X_train.shape[1])
    #autoencoder.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
    
    # Get reconstruction error
    #reconstructed = autoencoder.predict(X_test)
    #mse = np.mean(np.power(X_test - reconstructed, 2), axis=1)
    #threshold = np.percentile(mse, 90)
    #ae_pred = (mse > threshold).astype(int)
    #ae_metrics = evaluate_model(y_test, ae_pred, mse, "Autoencoder")
    #predictions_dict['Autoencoder'] = mse

    # 3. Autoencoder parameters tuning
    print("\nStarting autoencoder parameter tuning and training...")
    final_autoencoder, mse_scores, (precision, recall, f1, _) = train_and_evaluate_autoencoder(X_train, X_test, y_test)
    predictions_dict['Tuned_Autoencoder'] = mse_scores
    
    # 4. LOF
    print("\nTraining Local Outlier Factor with Tuning...")
    lof_model = train_lof_with_tuning(X_train)
    lof_pred = lof_model.predict(X_test)
    lof_pred = (lof_pred == -1).astype(int)
    lof_scores = -lof_model.score_samples(X_test)
    lof_metrics = evaluate_model(y_test, lof_pred, lof_scores, "LOF with Tuning")
    predictions_dict['LOF_Tuned'] = lof_scores
    
    # Plot ROC curves
    print("\nPlotting ROC curves...")
    plot_roc_curves(y_test, predictions_dict)
    
    # Create comparison DataFrame
    models = ['XGBoost', 'Naive Bayes', 'Autoencoder', 'LOF']
    metrics = [xgb_metrics[:3], nb_metrics[:3], ae_metrics[:3], lof_metrics[:3]]
    
    comparison_df = pd.DataFrame(metrics, 
                               columns=['Precision', 'Recall', 'F1'],
                               index=models)
    
    # Plot comparison
    plt.figure(figsize=(10,6))
    comparison_df.plot(kind='bar')
    plt.title('Model Performance Comparison')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    return comparison_df, {
        'xgboost': xgb_model,
        'naive_bayes': nb_model,
        'autoencoder': autoencoder,
        'lof': lof_model
    }

In [ ]:
# Load the data
file_path = "../data/processed/imputedData.csv"
data = pd.read_csv(file_path)
results, models = trainAndDetect(data)
print("\nFinal Model Comparison:")
print(results)